In [ ]:
# working 3 / 12


just_len = 60

import re
import collections

NUM     = r'(?P<NUM>\d+)'
PLUS    = r'(?P<PLUS>\+)'
MINUS   = r'(?P<MINUS>-)'
LPAREN  = r'(?P<LPAREN>\()'
RPAREN  = r'(?P<RPAREN>\))'
WS      = r'(?P<WS>\s+)'

master_pattern = re.compile('|'.join((NUM, PLUS, MINUS, LPAREN, RPAREN, WS)))
Token = collections.namedtuple('Token', ['type', 'value'])

def generate_tokens(pattern, text):
    scanner = pattern.scanner(text)
    for m in iter(scanner.match, None):
        token = Token(m.lastgroup, m.group())

        if token.type != 'WS':
            yield token

class Node:
    def __init__(self, type, value=None):
        self.type = type
        self.value = value
        self.left = None
        self.right = None

class ExpressionEvaluator:

    def parse(self, text):
        self.tokens = generate_tokens(master_pattern, text)
        self.current_token = None
        self.next_token = None
        self._advance()

        # expr is the top level grammar. So we invoke that first.
        # it will invoke other functions to consume tokens according to grammar rules.
        return self.expr()

    def _advance(self):
        self.current_token, self.next_token = self.next_token, next(self.tokens, None)

    def _accept(self, token_type):
        # if there is a next token and the token type matches
        if self.next_token and self.next_token.type == token_type:
            self._advance()
            return True
        else:
            return False

    def _expect(self, token_type):
        if not self._accept(token_type):
            raise SyntaxError('Expected ' + token_type)

    def expr(self):
        expr_value = self.factor() # left of an operation
        while self._accept('PLUS') or self._accept('MINUS'):
            op = self.current_token.type

            right = self.factor() # right of operator
            op_node = Node(op)
            op_node.left = expr_value
            op_node.right = right
            expr_value = op_node

        return expr_value

  # Term is equivalent to a factor

    def factor(self):
        if self._accept('NUM'):
            return Node('NUM', self.current_token.value)
        elif self._accept('LPAREN'): # if it s "(" it expects an expression
            expr_value = self.expr()
            self._expect('RPAREN')
            return expr_value
        else:
            raise SyntaxError('Expect NUMBER or LPAREN')




expr ::= expr + factor | expr - factor | factor



factor::= (expr) | NUM

In [ ]:
# working 3 / 12

# helper function
# this is working print tree
def print_tree(root, type="type", left="left", right="right"):
    def display(root, type=type, left=left, right=right):
        if getattr(root, right) is None and getattr(root, left) is None:
            label = getattr(root, type)
            value = getattr(root, 'value', None)
            if value is not None:
                line = f'{label} {value}'
            else:
                line = label
            width = len(line)
            height = 1
            middle = width // 2
            return [line], width, height, middle

        if getattr(root, right) is None:
            lines, n, p, x = display(getattr(root, left))
            label = getattr(root, type)
            value = getattr(root, 'value', None)
            if value is not None:
                s = f'{label} {value}'
            else:
                s = label
            u = len(s)
            first_line = (x + 1) * ' ' + (n - x - 1) * '_' + s
            second_line = x * ' ' + '/' + (n - x - 1 + u) * ' '
            shifted_lines = [line + u * ' ' for line in lines]
            return [first_line, second_line] + shifted_lines, n + u, p + 2, n + u // 2

        if getattr(root, left) is None:
            lines, n, p, x = display(getattr(root, right))
            label = getattr(root, type)
            value = getattr(root, 'value', None)
            if value is not None:
                s = f'{label} {value}'
            else:
                s = label
            u = len(s)
            first_line = s + x * '_' + (n - x) * ' '
            second_line = (u + x) * ' ' + '\\' + (n - x - 1) * ' '
            shifted_lines = [u * ' ' + line for line in lines]
            return [first_line, second_line] + shifted_lines, n + u, p + 2, u // 2

        left, n, p, x = display(getattr(root, left))
        right, m, q, y = display(getattr(root, right))
        label = getattr(root, type)
        value = getattr(root, 'value', None)
        if value is not None:
            s = f'{label} {value}'
        else:
            s = label
        u = len(s)
        first_line = (x + 1) * ' ' + (n - x - 1) * '_' + s + y * '_' + (m - y) * ' '
        second_line = x * ' ' + '/' + (n - x - 1 + u + y) * ' ' + '\\' + (m - y - 1) * ' '
        if p < q:
            left += [n * ' '] * (q - p)
        elif q < p:
            right += [m * ' '] * (p - q)
        zipped_lines = zip(left, right)
        lines = [first_line, second_line] + [a + u * ' ' + b for a, b in zipped_lines]
        return lines, n + m + u, max(p, q) + 2, n + u // 2

    lines, *_ = display(root, type, left, right)
    for line in lines:
        print(line)

LL parsers build the tree from the top-down, while LR parsers build the tree from the bottom-up.

so this is a LR

In [ ]:
# working 3/12

e = ExpressionEvaluator()
result = e.parse('22 + ( 3 + 33) - 5 + (2 - 4) + 1')

print('Result:', result.type)
print('Parse Tree:')
print_tree(result)

Result: PLUS
Parse Tree:
                                      ________________PLUS__   
                                     /                      \  
                            _______PLUS_______            NUM 1
                           /                  \                
         ________________MINUS__          __MINUS__            
        /                       \        /         \           
    __PLUS_______             NUM 5    NUM 2     NUM 4         
   /             \                                             
NUM 22       __PLUS___                                         
            /         \                                        
          NUM 3    NUM 33                                      


In [ ]:
e = ExpressionEvaluator()
result = e.parse('22 + ( 3 + (44 - (33- 3))) - 5 + (2 - 4) + 1')

print('Result:', result)
print('Parse Tree:')
print_tree(result)

Result: <__main__.Node object at 0x7c29d20493c0>
Parse Tree:
                                                           ________________PLUS__   
                                                          /                      \  
                                                 _______PLUS_______            NUM 1
                                                /                  \                
         _____________________________________MINUS__          __MINUS__            
        /                                            \        /         \           
    __PLUS_______                                  NUM 5    NUM 2     NUM 4         
   /             \                                                                  
NUM 22       __PLUS________                                                         
            /              \                                                        
          NUM 3        __MINUS________                                              
    

In [ ]:
e = ExpressionEvaluator()
while True:
  print("Enter you expression (q to quit): ",end="")
  expresion = input()
  if expresion == "q":
    break
  result = e.parse(expresion)
  print('Parse Tree:')
  print_tree(result)
  print(),print(),print()

Enter you expression (q to quit): Parse Tree:
   __PLUS_______        
  /             \       
NUM 1       __MINUS__   
           /         \  
         NUM 2     NUM 3



Enter you expression (q to quit): 